In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import classification_report, confusion_matrix

from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.model_selection import StratifiedKFold

from sklearn.tree import DecisionTreeClassifier

# Preprocessing

In [2]:
data=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

In [3]:
# data.isna().sum()

In [4]:
lel=LabelEncoder()
lel.fit(data["Churn_risk"])
data["Churn_risk"]=lel.transform(data["Churn_risk"])


le=LabelEncoder()
le.fit(data["Gender"])
data["Gender"]=le.transform(data["Gender"])
test["Gender"]=le.transform(test["Gender"])

le=LabelEncoder()
le.fit(data["Marital_status"])
data["Marital_status"]=le.transform(data["Marital_status"])
test["Marital_status"]=le.transform(test["Marital_status"])

le=LabelEncoder()
le.fit(data["Department"])
data["Department"]=le.transform(data["Department"])
test["Department"]=le.transform(test["Department"])


In [5]:
#data.dropna(inplace=True)

In [6]:
y=np.array(data["Churn_risk"])
data.drop(["Churn_risk"],axis=1,inplace=True)

In [7]:
imputer = KNNImputer(n_neighbors=105)
imputer.fit(data)

test = pd.DataFrame(imputer.transform(test))
test.columns=data.columns
data = pd.DataFrame(imputer.transform(data))
data.columns=test.columns


Scale Data

In [8]:
data["Employee_ID"]=data["Employee_ID"].apply(lambda x :int(x))
test["Employee_ID"]=test["Employee_ID"].apply(lambda x :int(x))

In [9]:
scaler=MinMaxScaler()
#scaler=StandardScaler()
data.iloc[:,1:]=scaler.fit_transform(data.iloc[:,1:])
test.iloc[:,1:]=scaler.transform(test.iloc[:,1:])

In [10]:

data.set_index("Employee_ID",inplace=True)
test.set_index("Employee_ID",inplace=True)

# Final

In [11]:
to_drop=[]

In [12]:
modelTREE = DecisionTreeClassifier(ccp_alpha= 0.0, criterion= 'gini', max_depth= 10,
                                max_features= None, 
                                max_leaf_nodes= 70, min_impurity_decrease= 0.0,
                                min_samples_leaf= 20, min_samples_split=2, min_weight_fraction_leaf= 0.0, 
                                splitter= 'best').fit(X = data, y = y)


In [13]:
modelTREE.fit(X = data.drop(to_drop,axis=1), y = y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=10, max_features=None, max_leaf_nodes=70,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=20, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [14]:
# from sklearn.tree import export_graphviz
# import graphviz
# import pydotplus

# def plot_tree(model):
#     dot_data = export_graphviz(model,
#                                feature_names=data.drop(to_drop,axis=1).columns
#                               )
#     pydot_graph = pydotplus.graph_from_dot_data(dot_data)
#     pydot_graph.set_size('"200,40"')
#     return graphviz.Source(pydot_graph.to_string())

In [15]:
# plot_tree(modelTREE)

In [16]:
Guests=pd.DataFrame(test.index)
Guests=Guests.astype('int64')
fina=pd.DataFrame(modelTREE.predict(test.drop(to_drop,axis=1)))
fina=pd.DataFrame(lel.inverse_transform(fina))
fina.reset_index(inplace=True)
fina.drop('index',inplace=True,axis=1)
final=pd.concat([fina,Guests],axis=1)
final.set_index('Employee_ID',inplace=True)
final.rename(columns={0:"Churn_risk"},inplace=True)

//anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:289: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [17]:
final.to_csv("versions/m20190546_.csv")

# Grid Search

In [18]:
# modelTREE = DecisionTreeClassifier(ccp_alpha= 0.0, criterion= 'gini', max_depth= 9,
#                                 max_features= None, 
#                                 max_leaf_nodes= 70, min_impurity_decrease= 0.0,
#                                 min_samples_leaf= 150, min_samples_split=2, min_weight_fraction_leaf= 0.0, 

In [19]:
# to_drop=[#'Gender',
#          'Satis_team',
#          'Emails',
#          'Tenure',
#          'Kids',
#          'Department']

In [20]:
# parameter_space = {
#    # 'criterion':['gini'], 
#    # 'splitter':['best'],
#     'max_depth':[8,9,10],
#   #  'min_samples_split':[2],
#     'min_samples_leaf':[100,120,125,140,150],
#   #  'min_weight_fraction_leaf':[0],
#   #  'max_features':[None],
#    # 'ccp_alpha':[0.0],
#    # 'min_impurity_decrease':[0.0],
#     'max_leaf_nodes':[40,50,60,70,80,90]
# }

In [21]:
# model = DecisionTreeClassifier()

In [22]:
# clf = GridSearchCV(model, parameter_space,n_jobs=8,verbose=5,cv=10,scoring='f1_micro')
                   
# #clf.fit(data.drop(to_drop,axis=1),y)   

# clf.fit(data.drop(to_drop,axis=1),y)


In [23]:

# # Best parameter set
# print('------------------------------------------------------------------------------------------------------------------------')
# print('Best parameters found:\n', clf.best_params_)
# print('------------------------------------------------------------------------------------------------------------------------')

# # All results
# means = clf.cv_results_['mean_test_score']
# stds = clf.cv_results_['std_test_score']
# for mean, std, params in zip(means, stds, clf.cv_results_['params']):
#     if mean>0.7:
#         print("%0.3f (+/-%0.03f) for %r" % (mean, std , params))

# *** Best Class ***

In [24]:
to_drop=[]

In [25]:
modelTREE = DecisionTreeClassifier(ccp_alpha= 0.0, criterion= 'gini', max_depth= 10,
                                max_features= None, 
                                max_leaf_nodes= 70, min_impurity_decrease= 0.0,
                                min_samples_leaf= 20, min_samples_split=2, min_weight_fraction_leaf= 0.0, 
                                splitter= 'best').fit(X = data, y = y)


modelTREE.fit(X = data.drop(to_drop,axis=1), y = y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=10, max_features=None, max_leaf_nodes=70,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=20, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')